#### How to create dataframe (summary table) for bronze, silver & gold tables? 

| DataFrame	| Row_Count	| Distinct_Count	| Aggregated_Sessions	| Min_Sessions | Max_Sessions	| Avg_Sessions | Null_Sessions | 	Columns_Count |
|-----------|-----------|-----------------|---------------------|--------------|--------------|--------------|---------------|--------------|
| Silver_tbl_01	| 100	| 100	| 8370	| 3	| 195	| 83.7	| 0	| 13 |
| Silver_tbl_02	| 100	| 100	| 7654	| 8	| 191	| 76.54	| 0	| 11 |
| Gold_table	  | 200	| 200	| 16024	| 3	| 195	| 80.12	| 0	| 14 |

In [0]:
from pyspark.sql.functions import lit, col, sum

In [0]:
silver_tbl_01 = spark.read.csv("/Volumes/@azureadb/pyspark/unionby/company_level.csv", header=True, inferSchema=True)
silver_tbl_01 = silver_tbl_01.withColumn("session_id", col("session_id").cast("string"))

print("Column Names of Silver Table 1: \n", silver_tbl_01.columns)
print("\nNo of Columns in Silver Table 1: ", len(silver_tbl_01.columns))
print("\nTotal Rows in Silver Table 01: ", silver_tbl_01.count())

display(silver_tbl_01.limit(10))

Column Names of Silver Table 1: 
 ['start_date', 'product_url', 'category', 'default_group', 'source_target', 'cloud_flatform', 'session_id', 'session_name', 'status_name', 'status_type', 'sessions', 'product_id', 'load datetime']

No of Columns in Silver Table 1:  13

Total Rows in Silver Table 01:  100


start_date,product_url,category,default_group,source_target,cloud_flatform,session_id,session_name,status_name,status_type,sessions,product_id,load datetime
2025-08-25,shop.sony.bpl,mobile,wifi-network,google,azure / aws / gcc,9876543,first_visit,first_visit,Not Available,5,409516064,2025-09-02T19:10:35
2025-08-26,shop.sony.bpl,mobile,wifi-network,google,azure / aws / gcc,9876544,purchase,organic,Not Available,12,409516064,2025-09-02T19:10:36
2025-08-27,shop.sony.bpl,mobile,wifi-network,google,azure / aws / gcc,9876545,search,network,Not Available,16,409516064,2025-09-02T19:10:37
2025-08-28,shop.sony.bpl,mobile,wifi-network,google,azure / aws / gcc,9876546,search,scroll,Not Available,22,409516064,2025-09-02T19:10:38
2025-08-29,shop.sony.bpl,mobile,wifi-network,google,azure / aws / gcc,9876547,search,organic,Not Available,25,409516064,2025-09-02T19:10:39
2025-08-30,shop.sony.bpl,mobile,wifi-network,(not set),azure / aws / gcc,9876548,add_to_cart,organic,Not Available,4,409516064,2025-09-02T19:10:40
2025-08-31,shop.sony.bpl,mobile,wifi-network,(not set),azure / aws / gcc,9876549,add_to_cart,organic,Not Available,9,409516064,2025-09-02T19:10:41
2025-09-01,shop.sony.bpl,mobile,wifi-network,(none) / (direct),azure / aws / gcc,9876550,add_to_cart,first_visit,Not Available,8,409516064,2025-09-02T19:10:42
2025-09-02,shop.sony.bpl,mobile,wifi-network,flipkart / referral,azure / aws / gcc,9876551,add_to_cart,first_visit,Not Available,7,409516064,2025-09-02T19:10:43
2025-09-03,shop.sony.bpl,mobile,wifi-network,(data not available),azure / aws / gcc,9876552,add_to_cart,first_visit,Not Available,6,409516064,2025-09-02T19:10:44


In [0]:
silver_tbl_02 = spark.read.csv("/Volumes/@azureadb/pyspark/unionby/device_level.csv", header=True, inferSchema=True)

print("Column Names of Silver Table 2: \n", silver_tbl_02.columns)
print("\nNo of Columns in Silver Table 2: ", len(silver_tbl_02.columns))
print("\nTotal Rows in Silver Table 02: ", silver_tbl_02.count())

display(silver_tbl_02.limit(10))

Column Names of Silver Table 2: 
 ['start_date', 'product_url', 'category', 'default_group', 'source_target', 'cloud_flatform', 'status_name', 'status_type', 'sessions', 'product_id', 'load datetime']

No of Columns in Silver Table 2:  11

Total Rows in Silver Table 02:  100


start_date,product_url,category,default_group,source_target,cloud_flatform,status_name,status_type,sessions,product_id,load datetime
2025-08-25,shop.sony.bpl,mobile,wifi-network,google,azure / aws / gcc,first_visit,Not Available,55,409516064,2025-09-02T19:10:35
2025-08-26,shop.sony.bpl,mobile,wifi-network,google,azure / aws / gcc,organic,Not Available,12,409516064,2025-09-02T19:10:36
2025-08-27,shop.sony.bpl,mobile,wifi-network,google,azure / aws / gcc,network,Not Available,16,409516064,2025-09-02T19:10:37
2025-08-28,shop.sony.bpl,mobile,wifi-network,google,azure / aws / gcc,scroll,Not Available,22,409516064,2025-09-02T19:10:38
2025-08-29,shop.sony.bpl,mobile,wifi-network,google,azure / aws / gcc,organic,Not Available,25,409516064,2025-09-02T19:10:39
2025-08-30,shop.sony.bpl,mobile,wifi-network,(not set),azure / aws / gcc,organic,Not Available,41,409516064,2025-09-02T19:10:40
2025-08-31,shop.sony.bpl,mobile,wifi-network,(not set),azure / aws / gcc,organic,Not Available,91,409516064,2025-09-02T19:10:41
2025-09-01,shop.sony.bpl,mobile,wifi-network,(none) / (direct),azure / aws / gcc,first_visit,Not Available,28,409516064,2025-09-02T19:10:42
2025-09-02,shop.sony.bpl,mobile,wifi-network,flipkart / referral,azure / aws / gcc,first_visit,Not Available,17,409516064,2025-09-02T19:10:43
2025-09-03,shop.sony.bpl,mobile,wifi-network,(data not available),azure / aws / gcc,first_visit,Not Available,16,409516064,2025-09-02T19:10:44


In [0]:
diff_cols_silver_1_2 = set(silver_tbl_01.columns) - set(silver_tbl_02.columns)
print("Columns in Silver Table 1 but not in Silver Table 2: ", diff_cols_silver_1_2)

Columns in Silver Table 1 but not in Silver Table 2:  {'session_name', 'session_id'}


In [0]:
# step:1 adding GRANULARITY column to identify the level of dataframe
df_silver_tbl_01 = silver_tbl_01.withColumn("GRANULARITY", lit("campaign"))

df_silver_tbl_02 = silver_tbl_02 \
    .withColumn("session_id", lit("NULL")) \
    .withColumn("session_name", lit("NULL")) \
    .withColumn("GRANULARITY", lit("device_category"))

# step:2 union of silver dataframes
df_gold_tbl = df_silver_tbl_01.unionByName(df_silver_tbl_02)
display(df_gold_tbl)

start_date,product_url,category,default_group,source_target,cloud_flatform,session_id,session_name,status_name,status_type,sessions,product_id,load datetime,GRANULARITY
2025-08-25,shop.sony.bpl,mobile,wifi-network,google,azure / aws / gcc,9876543,first_visit,first_visit,Not Available,5,409516064,2025-09-02T19:10:35,campaign
2025-08-26,shop.sony.bpl,mobile,wifi-network,google,azure / aws / gcc,9876544,purchase,organic,Not Available,12,409516064,2025-09-02T19:10:36,campaign
2025-08-27,shop.sony.bpl,mobile,wifi-network,google,azure / aws / gcc,9876545,search,network,Not Available,16,409516064,2025-09-02T19:10:37,campaign
2025-08-28,shop.sony.bpl,mobile,wifi-network,google,azure / aws / gcc,9876546,search,scroll,Not Available,22,409516064,2025-09-02T19:10:38,campaign
2025-08-29,shop.sony.bpl,mobile,wifi-network,google,azure / aws / gcc,9876547,search,organic,Not Available,25,409516064,2025-09-02T19:10:39,campaign
2025-08-30,shop.sony.bpl,mobile,wifi-network,(not set),azure / aws / gcc,9876548,add_to_cart,organic,Not Available,4,409516064,2025-09-02T19:10:40,campaign
2025-08-31,shop.sony.bpl,mobile,wifi-network,(not set),azure / aws / gcc,9876549,add_to_cart,organic,Not Available,9,409516064,2025-09-02T19:10:41,campaign
2025-09-01,shop.sony.bpl,mobile,wifi-network,(none) / (direct),azure / aws / gcc,9876550,add_to_cart,first_visit,Not Available,8,409516064,2025-09-02T19:10:42,campaign
2025-09-02,shop.sony.bpl,mobile,wifi-network,flipkart / referral,azure / aws / gcc,9876551,add_to_cart,first_visit,Not Available,7,409516064,2025-09-02T19:10:43,campaign
2025-09-03,shop.sony.bpl,mobile,wifi-network,(data not available),azure / aws / gcc,9876552,add_to_cart,first_visit,Not Available,6,409516064,2025-09-02T19:10:44,campaign


In [0]:
df_gold_tbl.select("session_id").distinct().display()

session_id
9876568
9876557
9876592
9876582
9876567
9876616
9876569
9876549
9876597
9876556


In [0]:
df_gold_tbl.select("session_name").distinct().display()

session_name
add_to_cart
first_visit
purchase
click
search
session_start
NULL


In [0]:
from pyspark.sql.functions import sum, min, max, avg, col

data = [
    [
        "Silver_tbl_01",
        silver_tbl_01.count(),
        silver_tbl_01.distinct().count(),
        silver_tbl_01.groupBy().agg(sum("sessions")).collect()[0][0],
        silver_tbl_01.agg(min("sessions")).collect()[0][0],
        silver_tbl_01.agg(max("sessions")).collect()[0][0],
        silver_tbl_01.agg(avg("sessions")).collect()[0][0],
        silver_tbl_01.filter(col("sessions").isNull()).count(),
        len(silver_tbl_01.columns)
    ],
    [
        "Silver_tbl_02",
        silver_tbl_02.count(),
        silver_tbl_02.distinct().count(),
        silver_tbl_02.groupBy().agg(sum("sessions")).collect()[0][0],
        silver_tbl_02.agg(min("sessions")).collect()[0][0],
        silver_tbl_02.agg(max("sessions")).collect()[0][0],
        silver_tbl_02.agg(avg("sessions")).collect()[0][0],
        silver_tbl_02.filter(col("sessions").isNull()).count(),
        len(silver_tbl_02.columns)
    ],
    [
        "Gold_table",
        df_gold_tbl.count(),
        df_gold_tbl.distinct().count(),
        df_gold_tbl.groupBy().agg(sum("sessions")).collect()[0][0],
        df_gold_tbl.agg(min("sessions")).collect()[0][0],
        df_gold_tbl.agg(max("sessions")).collect()[0][0],
        df_gold_tbl.agg(avg("sessions")).collect()[0][0],
        df_gold_tbl.filter(col("sessions").isNull()).count(),
        len(df_gold_tbl.columns)
    ]
]

df_tst = spark.createDataFrame(
    data, 
    ["DataFrame", "Row_Count", "Distinct_Count", "Aggregated_Sessions",
     "Min_Sessions", "Max_Sessions", "Avg_Sessions", "Null_Sessions", "Columns_Count"]
)

display(df_tst)

DataFrame,Row_Count,Distinct_Count,Aggregated_Sessions,Min_Sessions,Max_Sessions,Avg_Sessions,Null_Sessions,Columns_Count
Silver_tbl_01,100,100,8370,3,195,83.7,0,13
Silver_tbl_02,100,100,7654,8,191,76.54,0,11
Gold_table,200,200,16024,3,195,80.12,0,14
